In [2]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage import io, color
import cv2
import os

# Function definitions

In [31]:
# Based on tutorial: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
def make_square(img, desired_size=256, fill_color=[255, 255, 255]):
    if img.dtype != np.uint8:
        print(f'Converting to uint8...')
        img = (255*img).astype(np.uint8)
        
    scale_factor = desired_size/max(img.shape[0], img.shape[1])
    resized = cv2.resize(img, (int(scale_factor*img.shape[1]), int(scale_factor*img.shape[0])))
    new_size = resized.shape
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    out = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=fill_color)
    return out

# use float for thresholding, but return uint8 image
def binarize(img, threshold=0.5, invert=True):
    if img.dtype == np.uint8:
        img = img/255.0 # convert to float64

    # Convert to grayscale
    if len(img.shape) >= 3:
        img = color.rgb2gray(img)
    
    # Threshold
    out = np.zeros_like(img)
    if invert: # detect dark characters
        mask = img < threshold
    else: # detect light characters
        mask = img > threshold
    out[mask] = 1
    return (255*out).astype(np.uint8)

def preprocess(img, desired_size=256, fill_color=[255, 255, 255], threshold=0.5, invert=True):
    img_square = make_square(img, desired_size=desired_size, fill_color=fill_color)
    img_bin = binarize(img_square, threshold=threshold, invert=invert)
    return img_bin

def contour_analysis(img, n=2, trim_points=10, bins=20, verbose=False):
    # Find all contours
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if verbose:
        print(f'# of contours: {len(contours)}')
        print('# of points in each contour:')
        for cnt in contours:
            print(f'\t{len(cnt)} points')

    # Remove contours with too few points
    contours_trimmed = [cnt for cnt in contours if len(cnt) > trim_points]
    if verbose:
        print(f'Trimming contours with fewer than {trim_points} points...')
        print(f'# of remaining contours: {len(contours_trimmed)}')
        for cnt in contours_trimmed:
            print(f'\t{len(cnt)} points')

    # Create dashed contours by keeping every nth point
    assert(n>=2)
    contours_dashed = [cnt[1::n] for cnt in contours_trimmed]
    if verbose:
        print(f'Taking every {n}th point to get dashed contour...')
        for cnt in contours_dashed:
            print(f'\t{len(cnt)} points')
            
    # Find angles between adjacent points in the contour
    thetaseq = []
    for i, cnt in enumerate(contours_dashed):
        for j, point in enumerate(cnt):
            if j == 0:
                prevx, prevy = point[0]
            else:
                x, y = point[0]
                thetaseq.append(np.arctan2(y-prevy, x-prevx))
                prevx = x
                prevy = y
    
    if verbose:
        print(f'# thetas: {len(thetaseq)}')
        print(f'max theta: {max(thetaseq)}')
        print(f'min theta: {min(thetaseq)}')

    hist =  np.histogram(thetaseq, bins=bins, range=(-np.pi, np.pi))
    return hist

def plot_angles(hist, ax=None, title='Distribution of angles'):
    r, theta = hist
    r = np.append(r, 0) # append zero to correspond to the last angle

    if ax is None:
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(4, 4), dpi=300)

    ax.plot(theta, r)
    ax.grid(True)

    ax.set_title(title, va='bottom')
    plt.tight_layout()
    
    
def plot_mu_and_K(X, title='', path=''):
    mu = np.mean(X, axis=1)
    K = np.cov(X)
    bins = len(mu)
    theta = np.linspace(-np.pi, np.pi, bins+1)
 
    fig = plt.figure(figsize=(8, 4))
    ax1 = plt.subplot(121, projection='polar')
    ax2 = plt.subplot(122)
    plot_angles([mu, theta], ax=ax1, title='Mean vector μ')
    fig.colorbar(ax2.imshow(K))
    ax2.set_title('Covariance matrix K')
    plt.suptitle(title, fontsize='xx-large')
    plt.tight_layout()
    plt.savefig(path + 'mu+K.png')

# Mi Fu's *Poem Written In a Boat on the Wu River*

In [32]:
path = 'Extracted Characters/Mi Fu - Poem Written in a Boat on the Wu River/'
filenames = os.listdir(path)
filenames.sort()
print(filenames)

['001.png', '002.png', '003.png', '004.png', '005.png', '006.png', '007.png', '008.png', '009.png', '010.png', '011.png', '012.png', '013.png', '014.png', '015.png', '016.png', '017.png', '018.png', '019.png', '020.png', '021.png', '022.png', '023.png', '024.png', '025.png', '026.png', '027.png', '028.png', '029.png', '030.png', '031.png']


In [34]:
bins = 20
X = np.zeros((bins, len(filenames)))
for i, filename in enumerate(filenames):
    img = io.imread(path + filename)
    counts, theta = contour_analysis(preprocess(img), bins=bins)
    # divide counts by total counts to get Probability Mass Function (PMF)
    probs = counts / counts.sum()
    X[:, i] = probs
    
plot_mu_and_K(X, title=f'Wu River ({len(filenames)} images)', path='TestWuRiver/')

In [24]:
# Now call the function that automates this